# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: 
* Model: 
* Evaluation approach: 
* Fine-tuning dataset: 

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [1]:
from datasets import load_dataset

import torch

# Set the device to be used for the tensor
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# The sms_spam dataset only has a train split, so we use the train_test_split method to split it into train and test
dataset = load_dataset("sms_spam", split="train").train_test_split(
    test_size=0.2, shuffle=True, seed=23
)

splits = ["train", "test"]

# View the dataset characteristics
dataset["train"]

Generating train split:   0%|          | 0/5574 [00:00<?, ? examples/s]

Dataset({
    features: ['sms', 'label'],
    num_rows: 4459
})

In [2]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def preprocess_function(examples):
    """Preprocess the imdb dataset by returning tokenized examples."""
    return tokenizer(examples['sms'], padding='max_length', truncation=True, return_tensors = "pt")


tokenized_ds = {}

for split in splits:
    tokenized_ds[split] = dataset[split].map(preprocess_function, batched=True)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/4459 [00:00<?, ? examples/s]

Map:   0%|          | 0/1115 [00:00<?, ? examples/s]

In [3]:
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=2,
    id2label={0: "not spam", 1: "spam"},
    label2id={"not spam": 0, "spam": 1},
)

for param in model.base_model.parameters():
    param.requires_grad = False

# Check if the base model parameters are frozen
frozen = all([not param.requires_grad for param in model.base_model.parameters()])
print(f"Base model parameters frozen: {frozen}")

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Base model parameters frozen: True


In [4]:
import numpy as np
from transformers import DataCollatorWithPadding, Trainer, TrainingArguments


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}


# The HuggingFace Trainer class handles the training and eval loop for PyTorch for us.
# Read more about it here https://huggingface.co/docs/transformers/main_classes/trainer
trainer = Trainer(
    model=model,
    args=TrainingArguments(
        output_dir="./data/base_model_class_train",
        learning_rate=2e-3,
        # Reduce the batch size if you don't have enough memory
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        num_train_epochs=1,
        weight_decay=0.01,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
    ),
    train_dataset=tokenized_ds["train"].shuffle(seed=42).select(range(10)),
    eval_dataset=tokenized_ds["test"].shuffle(seed=42).select(range(10)),
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer = tokenizer),
    compute_metrics=compute_metrics,
)

trainer.evaluate()

/opt/conda/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32


/opt/conda/lib/python3.10/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.290810,0.900000


Checkpoint destination directory ./data/base_model_class_train/checkpoint-3 already exists and is non-empty.Saving will proceed but saved results may be invalid.


TrainOutput(global_step=3, training_loss=1.0553007125854492, metrics={'train_runtime': 39.2128, 'train_samples_per_second': 0.255, 'train_steps_per_second': 0.077, 'total_flos': 1324673986560.0, 'train_loss': 1.0553007125854492, 'epoch': 1.0})

In [5]:
trainer.evaluate()

{'eval_loss': 0.2908099889755249,
 'eval_accuracy': 0.9,
 'eval_runtime': 16.998,
 'eval_samples_per_second': 0.588,
 'eval_steps_per_second': 0.176,
 'epoch': 1.0}

In [6]:
inputs = tokenizer(dataset['test']['sms'][:10], truncation=True, padding='max_length', return_tensors="pt").to(device)
output = model(**inputs)

In [7]:
pred = np.argmax(output['logits'].detach().numpy(),1)
actual = dataset['test']['label'][:10]

In [8]:
acc = (pred == actual).mean()
print('The accuracy of baseline model is:', acc)

The accuracy of baseline model is: 1.0


## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [9]:
from peft import LoraConfig, TaskType,get_peft_model

In [10]:
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=2,
    id2label={0: "not spam", 1: "spam"},
    label2id={"not spam": 0, "spam": 1},
)

print(model)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [11]:
config = LoraConfig(target_modules=["v_lin","pre_classifier",
                                   "classifier"], task_type=TaskType.SEQ_CLS)
lora_model = get_peft_model(model, config)
lora_model.print_trainable_parameters()

trainable params: 1,294,884 || all params: 67,657,764 || trainable%: 1.9138734765163095


In [12]:
training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch",
                                 num_train_epochs=2)

trainer = Trainer(
    model=lora_model,
    args=training_args,
    train_dataset=tokenized_ds["train"].shuffle(seed=42).shuffle(seed=42).select(range(10)),
    eval_dataset=tokenized_ds["test"].shuffle(seed=42).shuffle(seed=42).select(range(10)),
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.600339,0.900000
2,No log,0.595276,0.900000


TrainOutput(global_step=4, training_loss=0.6899093985557556, metrics={'train_runtime': 121.1362, 'train_samples_per_second': 0.165, 'train_steps_per_second': 0.033, 'total_flos': 2692525178880.0, 'train_loss': 0.6899093985557556, 'epoch': 2.0})

In [13]:
trainer.evaluate()

{'eval_loss': 0.59527587890625,
 'eval_accuracy': 0.9,
 'eval_runtime': 17.3023,
 'eval_samples_per_second': 0.578,
 'eval_steps_per_second': 0.116,
 'epoch': 2.0}

In [14]:
lora_model.save_pretrained("distilbert-lora")

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [15]:
from peft import AutoPeftModelForSequenceClassification
lora_model = AutoPeftModelForSequenceClassification.from_pretrained("distilbert-lora",
                                                                    id2label={0: "not spam", 1: "spam"})

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
inputs = tokenizer(dataset['test']['sms'][:10], truncation=True, padding='max_length', return_tensors="pt")
output = lora_model(**inputs)

In [17]:
lora_pred = np.argmax(output['logits'].detach().numpy(),1)
actual = dataset['test']['label'][:10]

In [18]:
peft_acc = (lora_pred == actual).mean()
print('The accuracy of peft updated model is:', peft_acc)

The accuracy of peft updated model is: 1.0
